Как известно, тексты в этом датасете принадлежат к одной из трех категорий:
анекдоты
газета "Известия"
журнал "Техника молодежи"
Сначала нужно провести предобработку текстов: лемматизировать, удалить стоп-слова, возможно, разметить части речи (но последнее необязательно).
Для классификации нам нужно перейти от документов как текстовых строк к документам в виде набора чисел. Ранее мы с вами это делали при помощи модели bag-of-words и простых метрик типа длины слова и предложения в словах. Теперь давайте превратим документы в плотные вектора, созданные при помощи дистрибутивной семантики.
Вам нужно построить репрезентацию каждого текстового документа при помощи модели word2vec, натренированной на НКРЯ, с которой мы экспериментировали. Сделать это можно следующими способами:
самое простое -- сделать "семантический отпечаток" текста: посчитать средний вектор всех слов в этом тексте.
чуть сложнее, что мы не проходили, использовать doc2vec. Он реализован в библиотеке gensim, но нужно почитать документацию
ещё сложнее, это реализовать какой-нибудь алгоритм типа deep inverse regression
Вы можете выбрать любой из этих способов.


Затем при помощи scikitlearn вам нужно написать классификатор, который предсказывал бы класс текста на основе его вектора. Тестировать можно на тех же данных при помощи кросс-валидациию. Из алгоритмов лучше всего взять логистическую регрессию.
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
 
clf = LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=4)
В качестве оценки вашего классификатора посчитайте точность, полноту и F1-меру.

Итого результатом вашей работы должны быть:
скрипт, который переводит документы в векторную форму
скрипт, который тренирует на получившихся данных классификатор
метрики качества, которые говорит скрипт

In [309]:
import os, re
import pymorphy2
import gensim
import logging
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [310]:
classes = []
texts = []
dir_names = os.listdir('texts')
for d in dir_names:
    text_names = os.listdir('texts/' + d)
    for t in text_names:
        with open('texts/' + d + '/' + t, 'r', encoding='utf-8') as f:
            classes.append(d)
            texts.append(f.read())
df = pd.DataFrame()
df['class'] = classes
df['text'] = texts
df.head()

,class,text
0,anekdots,Абстрактный анекдот\n***\nЛетят два кирпича.\n...
1,anekdots,Андропов\n***\n-- Какую поэзию любит Андропов?...
2,anekdots,Англичане\n***\nПожилой джентльмен приучил себ...
3,anekdots,Аптекари\n***\nСтарый аптекарь оставил молодог...
4,anekdots,"Военные\n***\nВ армии нет вечных двигателей, в..."


In [325]:
def normalization(text):
    stops = set(stopwords.words('russian'))
    
    morph_analyzer = pymorphy2.MorphAnalyzer()
    tokenizer = TreebankWordTokenizer()
    
    tokenized = tokenizer.tokenize(text.lower())
    lemmatized = ['{}_{}'.format(morph_analyzer.parse(re.sub('[^\w]', '', word))[0].normal_form,
                                 morph_analyzer.parse(re.sub('[^\w]', '', word))[0].tag.POS)
                  for word in tokenized 
                  if re.sub('[^\w]', '', word) not in stops 
                  and re.sub('[^\w]', '', word) != '']
    
    return ' '.join(lemmatized)

In [326]:
df['text'] = df.text.apply(normalization)

2017-03-29 02:14:00,506 : INFO : Loading dictionaries from C:\Users\Maria\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-03-29 02:14:00,625 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2017-03-29 02:14:06,798 : INFO : Loading dictionaries from C:\Users\Maria\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-03-29 02:14:06,890 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2017-03-29 02:14:07,039 : INFO : Loading dictionaries from C:\Users\Maria\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-03-29 02:14:07,131 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2017-03-29 02:14:08,503 : INFO : Loading dictionaries from C:\Users\Maria\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-03-29 02:14:08,598 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2017-03-29 02:14:08,894 : INFO : Loading dictionaries from C:\Users\Maria\Anaconda3\lib\site-packages\pymorp

In [327]:
df = df.sample(frac=1)
df.head()

,class,text
304,teh_mol,сергей_NOUN казменко_NOUN защитник_NOUN шесть_...
358,teh_mol,вступление_NOUN рассказ_NOUN шкловский_ADJF ар...
360,teh_mol,герман_NOUN смирнов_NOUN ньютон_NOUN который_A...
50,anekdots,марксизм_NOUN ленинизм_NOUN самый_ADJF коротки...
15,anekdots,чукча_NOUN новый_ADJF чукчамосквич_NOUN встреч...


In [328]:
df.groupby(['class']).size()

class
anekdots    125
izvest      125
teh_mol     125
dtype: int64

In [329]:
docs = '\n'.join(df['text'])
with open('text.txt', 'w', encoding='utf-8') as w:
    w.write(docs)

In [330]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [331]:
docs = gensim.models.doc2vec.TaggedLineDocument('text.txt')
model = gensim.models.Doc2Vec(docs, 
                              size=300, 
                              window=15, 
                              min_count=1)
model.save('mymodel')

2017-03-29 02:18:39,098 : WARNING : Slow version of gensim.models.word2vec is being used
2017-03-29 02:18:39,104 : INFO : collecting all words and their counts
2017-03-29 02:18:39,108 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-03-29 02:18:39,397 : INFO : collected 40915 word types and 375 unique tags from a corpus of 375 examples and 363975 words
2017-03-29 02:18:39,398 : INFO : Loading a fresh vocabulary
2017-03-29 02:18:39,549 : INFO : min_count=1 retains 40915 unique words (100% of original 40915, drops 0)
2017-03-29 02:18:39,550 : INFO : min_count=1 leaves 363975 word corpus (100% of original 363975, drops 0)
2017-03-29 02:18:39,742 : INFO : deleting the raw counts dictionary of 40915 items
2017-03-29 02:18:39,744 : INFO : sample=0.001 downsamples 16 most-common words
2017-03-29 02:18:39,747 : INFO : downsampling leaves estimated 356962 word corpus (98.1% of prior 363975)
2017-03-29 02:18:39,749 : INFO : estimated required memory for 40915 

In [333]:
model = gensim.models.Doc2Vec.load('mymodel')

2017-03-29 02:38:38,363 : INFO : loading Doc2Vec object from mymodel
2017-03-29 02:38:38,810 : INFO : loading wv recursively from mymodel.wv.* with mmap=None
2017-03-29 02:38:38,813 : INFO : loading syn0 from mymodel.wv.syn0.npy with mmap=None
2017-03-29 02:38:38,914 : INFO : setting ignored attribute syn0norm to None
2017-03-29 02:38:38,916 : INFO : loading docvecs recursively from mymodel.docvecs.* with mmap=None
2017-03-29 02:38:38,918 : INFO : loading syn1neg from mymodel.syn1neg.npy with mmap=None
2017-03-29 02:38:39,060 : INFO : setting ignored attribute cum_table to None
2017-03-29 02:38:39,061 : INFO : loaded mymodel


In [334]:
vectors = [el for el in model.docvecs]

In [335]:
X_train, X_test, y_train, y_test = train_test_split(vectors, df['class'], test_size=0.2)

In [336]:
clf = LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=4, random_state=0)

In [337]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=300, multi_class='multinomial',
          n_jobs=4, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [338]:
print(classification_report(y_test, preds))

             precision    recall  f1-score   support

   anekdots       0.67      0.15      0.24        27
     izvest       0.47      0.32      0.38        28
    teh_mol       0.22      0.55      0.31        20

avg / total       0.48      0.32      0.31        75



In [339]:
scores = cross_val_score(clf, vectors, df['class'], cv=5)
print(scores.mean())

0.776
